# Problem Statement

We want to understand what types of topics (subject matter) comprise the content in our documents. We can use **topic modelling** - using statistical methods to discovering the abstract/latent “topics” from a particular corpus.

<div class="alert-success">
used as EDA, to find patterns</div>

## Load Data

### BBC News Dataset

We will load a BBC News dataset with documents divided between politics, entertainment, business, sport, and tech.

### BBC Sport Dataset
We will be loading in the BBC Sport news dataset. It is divided between 5 distinct sports topics - football, athletics, cricket, rugby and tennis.

Both datasets are courtesy of *D. Greene and P. Cunningham. "Practical Solutions to the Problem of Diagonal Dominance in Kernel Document Clustering", Proc. ICML 2006.* ([Link](http://mlg.ucd.ie/datasets/bbc.html))


In [1]:
import pandas as pd
from typing import List, Tuple

#### Define Function to Load in BBC Datasets

In [4]:
def load_bbc_corpus(directory: str, topics: List[str], num_docs: int)-> pd.DataFrame:
    articles: List[Tuple[str, str]] = [(f"../datasets/{directory}/{topic}/{str(i).zfill(3)}.txt", topic) 
                                       for i in range(1,num_docs + 1) for topic in TOPICS]
    data = []
    for article, topic in articles:
        with open(article, encoding="latin1") as article: # open each sports article
            content = article.read()
            data.append({"topic": topic, "text": content})

    # generate a dataframe
    df = pd.DataFrame(data)
    df.text = df.text.apply(lambda text: text.replace("\n", " "))
    return df

#### Load BBC Sport Corpus

In [5]:
TOPICS = ["football", "athletics", "cricket", "rugby", "tennis"]
sports_corpus_df = load_bbc_corpus("bbcsport", TOPICS, num_docs=100)
sports_corpus_df.head(5)

topic                                               text
0   football  Man Utd stroll to Cup win  Wayne Rooney made a...
1  athletics  Claxton hunting first major medal  British hur...
2    cricket  Hayden sets up Australia win  Second one-day i...
3      rugby  Hodgson shoulders England blame  Fly-half Char...
4     tennis  Henman overcomes rival Rusedski  Tim Henman sa...

#### Load in BBC News Corpus

In [6]:
documents = []
TOPICS = ["business", "sport", "entertainment", "tech", "politics"]
news_corpus_df = load_bbc_corpus("bbc", TOPICS, num_docs=350)
news_corpus_df.head()

topic                                               text
0       business  Ad sales boost Time Warner profit  Quarterly p...
1          sport  Claxton hunting first major medal  British hur...
2  entertainment  Gallery unveils interactive tree  A Christmas ...
3           tech  Ink helps drive democracy in Asia  The Kyrgyz ...
4       politics  Labour plans maternity pay rise  Maternity pay...

### Technique 1: Non-Negative Matrix Factorization

We can think of our corpus as a two-dimensional table - rows being the documents, and columns being the features (ie. in a count-based vectorizer, each column being a unique token).

In Non-Negative Matrix Factorization, we try to find two matrices `W` and `H`, that contain only nonnegative values and when multiplied together, will reconstruct `X`. 

We need to select a variable `K`, which is the number of components/topics we wish to use.

If we want to model topics for a $N \times M$ matrix `X`, where each value is non-negative, then NMD will produce a $K \times M$ matrix `H` and a $N \times K$ matrix `W`.
![NMF](https://raw.githubusercontent.com/ychennay/dso-560-nlp-text-analytics/main/images/nmf.png)

<div class="alert-success">
    <p>$X$, $W$: doc matrix, $H$: term matrix 
    <p>decompose big matrix into two small matrix,
    <p>performance is evaluated by reconstruction error (i.e. frobenius norm)
</div>

#### Step 1: Vectorize The Corpus

In [8]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(2,2),
                             min_df=0.01, max_df=0.4, stop_words="english")

news_vectorizer = TfidfVectorizer(ngram_range=(3,3), min_df=3,
                            max_df=0.4, stop_words="english")

X_sport, sports_terms = vectorizer.fit_transform(sports_corpus_df.text), vectorizer.get_feature_names_out()
X_news, news_terms = news_vectorizer.fit_transform(news_corpus_df.text), news_vectorizer.get_feature_names_out()
sport_tf_idf = pd.DataFrame(X_sport.toarray(), columns=sports_terms)
news_tf_idf = pd.DataFrame(X_news.toarray(), columns=news_terms)
print(f"News TF-IDF: {news_tf_idf.shape}")
# print(news_tf_idf.head(5))
print(f"Sports TF-IDF: {sport_tf_idf.shape}")
sport_tf_idf.head(5)

News TF-IDF: (1750, 3354)
Sports TF-IDF: (500, 866)


10 000m  10 days  10 minutes  100m champion  100m silver  12 august  \
0      0.0      0.0         0.0            0.0          0.0        0.0   
1      0.0      0.0         0.0            0.0          0.0        0.0   
2      0.0      0.0         0.0            0.0          0.0        0.0   
3      0.0      0.0         0.0            0.0          0.0        0.0   
4      0.0      0.0         0.0            0.0          0.0        0.0   

   12 march  12 months  12th man  13 march  ...  years ago  years time  \
0       0.0        0.0       0.0       0.0  ...   0.000000         0.0   
1       0.0        0.0       0.0       0.0  ...   0.000000         0.0   
2       0.0        0.0       0.0       0.0  ...   0.000000         0.0   
3       0.0        0.0       0.0       0.0  ...   0.000000         0.0   
4       0.0        0.0       0.0       0.0  ...   0.208396         0.0   

   yelena isinbayeva  york marathon  young players  younis khan  \
0                0.0            0.0       0.232726          0.0   
1                0.0            0.0       0.000000          0.0   
2                0.0            0.0       0.000000          0.0   
3                0.0            0.0       0.000000          0.0   
4                0.0            0.0       0.000000          0.0   

   yousuf youhana  yuvraj singh  zaheer khan  zurich premiership  
0             0.0           0.0          0.0                 0.0  
1             0.0           0.0          0.0                 0.0  
2             0.0           0.0          0.0                 0.0  
3             0.0           0.0          0.0                 0.0  
4             0.0           0.0          0.0                 0.0  

[5 rows x 866 columns]

#### Step 2: Fit NMF Model

In [9]:
nmf = NMF(n_components=5)
W_sport = nmf.fit_transform(X_sport)
H_sport = nmf.components_
print(f"Original shape of X sports is {X_sport.shape}")
print(f"Decomposed W sports matrix is {W_sport.shape}")
print(f"Decomposed H sports matrix is {H_sport.shape}")

Original shape of X sports is (500, 866)
Decomposed W sports matrix is (500, 5)
Decomposed H sports matrix is (5, 866)


/Users/antheayang/opt/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,


In [10]:
W_news = nmf.fit_transform(X_news)
H_news = nmf.components_
print(f"Original shape of X news is {X_news.shape}")
print(f"Decomposed W news matrix is {W_news.shape}")
print(f"Decomposed H news matrix is {H_news.shape}")

Original shape of X news is (1750, 3354)
Decomposed W news matrix is (1750, 5)
Decomposed H news matrix is (5, 3354)


/Users/antheayang/opt/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,


In [29]:
W_sport[0]

array([0.        , 0.        , 0.00342875, 0.        , 0.28323909])

<div class="alert-success">
the first doc, is most relevant to topic 5, which we dont know
</div>

In [31]:
len(H_sport[0])

866

<div class="alert-success">
# of features
</div>

In [13]:
H_sport[0]

array([0.00000000e+00, 0.00000000e+00, 1.35328665e-02, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.63475422e-02, 1.85228570e-02,
       1.86140201e-01, 5.40253654e-03, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 6.66914392e-04, 8.86128088e-03,
       0.00000000e+00, 0.00000000e+00, 1.57006491e-02, 0.00000000e+00,
       0.00000000e+00, 8.01347973e-03, 1.14182213e-04, 9.94593293e-03,
       0.00000000e+00, 4.68674892e-03, 4.93886213e-02, 0.00000000e+00,
       2.60130331e-02, 4.09727807e-02, 7.02371451e-02, 0.00000000e+00,
       5.58517464e-04, 0.00000000e+00, 0.00000000e+00, 8.40537202e-02,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.72671804e-01,
       0.00000000e+00, 0.00000000e+00, 2.71419589e-01, 0.00000000e+00,
       8.26660577e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       2.12903331e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

<div class="alert-success">
the first original feature, is not relevant to the unknown topic 1, the 3rd feature has some relationship to it
</div>

#### Step 3: Report Results For Each Topic

In [19]:
from typing import List
import numpy as np

def get_top_tf_idf_tokens_for_topic(H: np.array, feature_names: List[str], num_top_tokens: int = 5):
    """
    Uses the H matrix (K components x M original features) to identify for each
    topic the most frequent tokens.
    """
    for topic, vector in enumerate(H):
        print(f"TOPIC {topic}\n")
        total = vector.sum()
        top_scores = vector.argsort()[::-1][:num_top_tokens] # positions of sorted order
        token_names = list(map(lambda idx: feature_names[idx], top_scores))
        strengths = list(map(lambda idx: vector[idx] / total, top_scores))

    for strength, token_name in zip(strengths, token_names):
        print(f"\b{token_name} ({round(strength * 100, 1)}%)\n")
    print(f"=" * 50)

In [37]:
topic, vector = 0, H_sport[0]
num_top_tokens = 5
total = vector.sum()
top_scores = vector.argsort()[::-1][:num_top_tokens]
token_names = list(map(lambda idx: sport_tf_idf.columns[idx], top_scores))
strengths = list(map(lambda idx: vector[idx] / total, top_scores))

In [48]:
top_scores

array([528, 614, 702, 715, 106])

In [17]:
get_top_tf_idf_tokens_for_topic(H_sport, sport_tf_idf.columns.tolist(), 5)

TOPIC 0

new zealand (7.4%)

ricky ponting (1.6%)

sri lanka (1.4%)

stephen fleming (1.4%)

brett lee (1.4%)

TOPIC 1

south africa (7.9%)

michael vaughan (2.4%)

graeme smith (2.2%)

nicky boje (2.0%)

ab villiers (2.0%)

TOPIC 2

australian open (4.0%)

world number (2.7%)

davis cup (2.6%)

grand slam (2.4%)

french open (1.9%)

TOPIC 3

cross country (2.8%)

year old (2.1%)

lewis francis (1.9%)

european indoor (1.8%)

world record (1.6%)

TOPIC 4

champions league (3.1%)

world cup (3.1%)

manchester united (2.4%)

told bbc (1.7%)

fa cup (1.6%)



In [18]:
print(f"BBC News Topics:\n\n")
get_top_tf_idf_tokens_for_topic(H_news, news_tf_idf.columns.tolist(), 10)

BBC News Topics:


TOPIC 0

told bbc news (14.4%)

bbc news website (12.8%)

bbc news online (1.6%)

spokesman told bbc (1.1%)

personal digital assistants (0.8%)

new york state (0.7%)

university new york (0.7%)

senior vice president (0.6%)

vice president technology (0.6%)

digital music players (0.6%)

TOPIC 1

told bbc sport (17.6%)

coach andy robinson (2.5%)

31 year old (2.2%)

22 year old (2.1%)

referee jonathan kaplan (2.1%)

30 year old (2.0%)

cross country championships (1.6%)

england coach andy (1.6%)

coach bernard laporte (1.4%)

coach mike ruddock (1.4%)

TOPIC 2

told bbc radio (2.5%)

bbc radio today (1.8%)

mr blair said (1.8%)

radio today programme (1.8%)

leader michael howard (1.7%)

mr howard said (1.3%)

tory leader michael (1.3%)

tony blair said (0.9%)

said mr blair (0.8%)

bbc radio live (0.8%)

TOPIC 3

bbc world service (10.0%)

boyd technology correspondent (4.6%)

correspondent world bbc (4.6%)

service wgbh boston 

<div class="alert-success">
could find some patterns for the important features, yet not sure about the real topic that AI thinks
</div>

#### Get the Top Documents For Each Topic

We can also use the `W` matrix to grab top documents per topic (ie. the document that had the greatest percentage of of each topic).

In [26]:
import numpy as np
def get_top_documents_for_each_topic(W: np.array, documents: List[str], num_docs: int = 5):
    """
    Uses the W matrix (N docs x K components/topics) to identify for each
    topic the most top documents.
    """
    sorted_docs = W.argsort(axis=0)[::-1]
    top_docs = sorted_docs[:num_docs].T
    per_document_totals = W.sum(axis=1)
    for topic, top_documents_for_topic in enumerate(top_docs):
        print(f"Topic {topic}")
        for doc in top_documents_for_topic:
            score = W[doc][topic]
            percent_about_topic = round(score / per_document_totals[doc] * 100, 1)
            print(f"{percent_about_topic}%", documents[doc])
            print('---------------')
        print("=" * 50)

In [27]:
get_top_documents_for_each_topic(W_sport, sports_corpus_df.text.tolist())

Topic 0
100.0% Hayden sets up Australia win  Second one-day international, Christchurch Australia 314-6 (50 overs) beat New Zealand 208 (40.4 overs) by 106 runs  Ricky Ponting (53) and Damien Martyn (58) provided the main support for Hayden, who hit two sixes and 12 fours. They eventually totalled 314-6 and the game was as good as over when New Zealand slumped to 73-6 in reply, with Adam Gilchrist taking five catches. Daniel Vettori made a rapid 83 but they were all out for 208 in the 41st over. New Zealand must now win all three remaining matches to take the series, but such a turn-round looked a remote possibility at Jade Stadium. Skipepr Stephen Fleming chose to put Australia in after winning the toss, with former rugby star Jeff Wilson included in the New Zealand team for his first one-day international since March 1993. He was taught a harsh lesson as his six overs cost 57 runs, with only left-arm spinner Vettori (0-31 from 10) able to exercise any measure of control over the Aust

In [28]:
get_top_documents_for_each_topic(W_news, news_corpus_df.text.tolist(), num_docs=10)

Topic 0
100.0% Tech helps disabled speed demons  An organisation has been launched to encourage disabled people to get involved in all aspects of motorsport, which is now increasingly possible thanks to technological innovations.  The Motorsport Endeavour Club left the starting grid yesterday at the Autosport International 2005 show at Birmingham's NEC, with several technologies to adapt vehicles on display.  Motorcycle racer, Roy Tansley, from Derby developed his electronic sequential gear changer following an accident which resulted in part of his left leg being amputated. "I needed to find a way of changing gear and generally you do that with your left leg," Mr Tansley told the BBC News website. "In simple terms, I needed to invent a left foot - initially it was quite a Heath Robinson device." Mr Tansley had to argue his case to be allowed to continue competing with motorcycle racing's governing body, the Autocycle Union. "At that time they wouldn't let any amputee race at all, but 

### Approach 2: LSA (Latent Semantic Analysis)

We can also leverage a dimensionality reduction technique that we've ecnountered before - **Singular Value Decomposition (SVD)** to perform topic modelling.

The following diagram and code snippet is from *Blueprints for Text Analytics Using Python*, Albrecht et al.

Remember that for SVD, we can take our original matrix and decompose it into three matrices.

$$
V = U \times \Sigma \times V^{\star}
$$
We can use these three decomposed matrices for different purposes ([Adel Amadyan](https://adel.ac/singular-value-decomposition-in-computer-vision/#Singular_Value_Decomposition182)):
![SVD Topic Modelling](https://i1.wp.com/adel.ac/wp-content/uploads/2019/02/svd.png?resize=500%2C200) 

The $U$ matrix will provide a signal for what the topic composition of our documents are.

The diagonal elements of the $\Sigma$ matrix can be used to estimate the "strength" of each topic. 

The $V^{star}$ matrix can be used to find the top associated words with each topic.

<div class="alert-success">
as SVD </div>

In [24]:
from sklearn.decomposition import TruncatedSVD

# we need to select a K (the number of topics)
K = 5

svd = TruncatedSVD(n_components=K)
U = svd.fit_transform(X_news)
V_star = svd.components_

In [25]:
print(f"U shape is {U.shape}")
get_top_documents_for_each_topic(U, news_corpus_df.text.tolist())

U shape is (1750, 5)
Topic 0
111.1% Games help you 'learn and play'  'God games' in which players must control virtual people and societies could be educational, says research.  A US researcher has suggested that games such as The Sims could be a good way to teach languages. Ravi Purushotma believes that the world of The Sims can do a better job of teaching vocabulary and grammar than traditional methods. The inherent fun of game playing could help to make learning languages much less of a chore, said Mr Purushotma.  There must be few parents or teachers that do not worry that the lure of a video game on a computer or console is hard to resist by children that really should be doing their homework. But instead of fearing computer games, Ravi Purushotma believes that educationalists, particularly language teachers should embrace games. "One goal would be to break what I believe to be the false assumption that learning and play are inherently oppositional," he said. He believes that the 

### Approach 3: Latent Dirichlet Allocation

The final approach uses a probablistic sampling method by viewing each document as consistenting of mixture of different topics, which are themselves mixtures of different words. 

Each of the mixtures (documents as mixtures of topics, topics as mixtures of words) are modelled using a [Dirichlet distribution](https://en.wikipedia.org/wiki/Dirichlet_distribution).

The algorithm creates initial Dirichlet distributions from each topic and word and tries to recreate the original words used for a document using sampling.

It first attempts to construct the representative words for a topic ([A Beginner's Guide to Latent Dirichlet Allocation, Ria Kulshrestha](https://towardsdatascience.com/latent-dirichlet-allocation-lda-9d1cd064ffa2)):
![https://miro.medium.com/max/1222/1*NjeMT281GMduRYvPIS8IjQ.png](https://miro.medium.com/max/1222/1*NjeMT281GMduRYvPIS8IjQ.png)

The algorithm looks like this:

![LDA](https://miro.medium.com/max/494/1*VTHd8nB_PBsDtd2hd87ybg.png) 

(from [LDA Topic Modeling: An Explanation](https://towardsdatascience.com/lda-topic-modeling-an-explanation-e184c90aadcd))

* $\alpha$ is the per-document topic distributions
* $\phi$ is the word distribution for a given topic
* $\beta$ is the per-topic word distributions
* $\theta$ is the topic distribution for the m-th document
* $Z$ is the topic assigned to the n-th word of the m-th document

We can only observe $W$. β is the table in the above screenshot (each row is a topic, each column is a word). 

We randomly initialize the initial topic distribution, and update iteratively until we converge to a solution or exceed the maximum number of iterations.

The New York Times [highlighed an example of a recommendation system based off of LDA](https://open.blogs.nytimes.com/2015/08/11/building-the-next-new-york-times-recommendation-engine/).

#### Assumptions of LDA
* Bag of words - each document is a bag of words where sequence, part of speech, etc. are not considered. 
* The number of topics is pre-determined and known (or guesstimated).

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=5)
W = lda.fit_transform(X_sport)
get_top_documents_for_each_topic(W, sports_corpus_df.text.tolist(), 10)

Topic 0
89.3% Flintoff fit to bowl at Wanderers  Fourth Test, Wanderers: South Africa v England  Plays starts Thursday, 0830 GMT  There had been concerns his rib muscle injury would restrict him to playing as a specialist batsman in the match. Captain Michael Vaughan said: "He's had a bowl and came out fine so he is fully fit to play as an all-rounder. "We will see how he bowls. In Cape Town I thought he was our best bowler and in Durban probably our second best." Flintoff sent down around 20 deliveries at three-quarter pace during Wednesday's practice session. The 27-year-old incurred a side strain during the 196-run defeat in Cape Townlast week and did not bowl again until the eve of the Johannesburg Test. Vaughan said he would not necessarily shield Flintoff from bowling the same heavy workload he endured in the first three Tests. The skipper commented: "We will just have to judge who is bowling well on any given day and on the given surface to see how much we use him.  "But as a fa

### Collaborative Filters

We can also leverage NLP models as part of a collaborative filter in order to generate product recommendations for a given user/customer.

A common approach is constructing an **User-Item** Iteraction Matrix, where there are many sparse elements. We can then iteratively fill compute the User and Item matrices that will minimize the least square error. This class of algorithms is called **Alternating Least Square**.
![https://miro.medium.com/max/1400/1*xMxQL_V9CWeLggrk-Uyzmg.png](https://miro.medium.com/max/1400/1*xMxQL_V9CWeLggrk-Uyzmg.png).

See here for an [example of using collaborative filters in Spark](https://gist.github.com/ychennay/0dc72d5098aa209985feed1a6b4f6b96).

[Himanshu Kriplani, "Alternating Least Square for Implicit Dataset with code"](https://towardsdatascience.com/alternating-least-square-for-implicit-dataset-with-code-8e7999277f4b). 

#### Appendix: Mathematical Theory for Non-Negative Matrix Factorization

Derivations are from [Source Separation Tutorial Mini-Series II: Introduction to Non-Negative Matrix Factorization](https://ccrma.stanford.edu/~njb/teaching/sstutorial/part2.pdf).

We attempt to minimize the divergence $D$, between the original matrix $X$ and the product of the deconstructed $W$ and $H$ matrices:
$$
min(D(V||\hat{V}))
$$
For NMF, this means
$$
min_{W, H >= 0}(D(V||W\times H))
$$
This is read as *we want to select non-negative values for $W$ and $H$ that will minimize $D$*.

There are many functions we can choose to approximate $D$ for examle, **Euclidean Distance**:
$$
D(V||\hat{V}) = \sum_{i,j}{(V_{ij} - \hat{V}_{ij})^{2}}
$$
However, in practice, we commonly select **[Kullback-Leibler Divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence)** to use for the divergence metric:      
$$
D(V||\hat{V}) = \sum_{i,j}{(V_{ij}log(\frac{V_{ij}}{\hat{V}_{ij}}) - V_{ij} + \hat{V}_{ij})}
$$
We can rewrite this as (by substituting $V_{ij}$ with $W\times H$):
$$
D(V||\hat{V}) = \sum_{i,j}{(V_{ij}log(\frac{V_{ij}}{W\times H}) - V_{ij} + W\times H)}
$$

From here, we can use [Jensen's Inequality](https://en.wikipedia.org/wiki/Jensen%27s_inequality) to rewrite this as 
$$
H^{\star}_{kj} = \frac{\sum{V_{ij}\pi_{ijk}}}{\sum{W_{ik}}}
$$
Here, $\pi_{ijk}$ is how much of the component $k$ to assign to the i-th document's j-th feature.
